![Erudio logo](../../img/erudio-logo-small.png)

# Context Managers

A context manager, as the name suggests, provides a certain *context* to a series of operations in Python.  Whenever you use the `with` statement in Python, that introduces a context manager.  The idea in a context manager is that some initial setup is performed to prepare for your custom code.  A context object is often provided to your code that incorporates something about that context, and provides a name for you to utilize.  After the block of code nested under `with` complete, some cleanup actions can be performed.  For example, file handles and socket connections can be closed, or configuration settings can be restored to their defaults.

# Standard library context managers

There are many context managers you are already using as part of your typical Python programs. Let us look at a few examples as a reminder.

In [1]:
# Create a scratch dirctory
from pathlib import Path
Path('tmp').mkdir(exist_ok=True)

# Preferred as the style of file access in modern Python
with open('tmp/advPython_test', 'w') as fw:
    fw.write("Hello")

with open('tmp/advPython_test', 'r') as fr:
    print(fr.read())

Hello


In [2]:
fh = open('tmp/advPython_test')
fh.close()

If `open()` did not provide a context manager (as indeed it did not in early versions of Python), we would have to write slightly more code; but more importantly, it is code that is easy to forget or to get subtly wrong when more is going on inside the code block in the context manager.

```python
with open('tmp/advPython_test', 'w') as fi:
    fi.write("Hello")
```

versus:

```python
try:
    fi = open('tmp/advPython_test', 'w')
    fi.write("Hello")
finally:
    fi.close()
```

Many other standard Python objects and classes likewise make themselves available as context managers.  The classes `zipfile.ZipFiles`, `subprocess.Popen`, `tarfile.TarFile`, `telnetlib.Telnet`, and `pathlib.Path` can be used as context managers. Or, for example, `urllib`:

In [3]:
import os
os.environ['SSL_CERT_DIR'] = '/etc/ssl/certs'

In [4]:
from urllib.request import urlopen
url = "https://courses.ine.com/area/data-science/"
with urlopen(url) as page:
    print(page.read(45))

b'<!DOCTYPE html>\n<html lang="en">\n   <head>\n  '


The `threading` module defines a variety of objects that may be used as context managers.  The simplest is a basic lock.

In [5]:
# Print several counts with different starting points in several threads
import threading

def worker(start):
    # Some large, parallelizable computation might live here
    for n in range(start, start+5):
        print("%d " % n, end='', flush=True)   
    print(' +\n', end='', flush=True)
    # Some additional large, parallelizable computation here

    
for i in range(0, 50, 10):
    t = threading.Thread(target=worker, args=(i,))
    t.start()

0 1 2 3 4  +
10 20 21 22 23 24 11 30  +
12 40 13 31 41 32 42 14 33 43 34  +
44  +
 +


The non-lock version is not wrong per-se, but suppose you'd like deterministic order to the outputs:

In [6]:
lock = threading.Lock()

def worker(start):
    # Some large, parallelizable computation might live here
    with lock:
        for n in range(start, start+5):
            print("%d " % n, end='', flush=True)
    print(' +\n', end='', flush=True)
    # Some additional large, parallelizable computation here
    
for i in range(0, 50, 10):
    t = threading.Thread(target=worker, args=(i,))
    t.start()

0 1 2 3 4  +
10 11 12 13 14  +
20 21 22 23 24  +
30 31 32 33 34  +
40 41 42 43 44  +


# Writing context managers

Much as Python provides an Iterator Protocol, it provides a Context Manager Protocol.  

A context managers is simply a class with two magic methods: `.__enter__()` and `.__exit__()`.  The purpose of these managers is to factor out often used try/finally clauses to make the code more readable.  As we have seen from several examples in the standard library, these same classes may also offer many other methods to exercise their capabilities in ways other than via context managers. In fact, `.__enter__()` and `.__exit__()` are often simply synonyms for more conventionally named methods like `.open()` and `.close()`.

In [7]:
class MyContext(object):
    def __init__(self, val=42):
        self.val = val
        print("Initializing context")
        
    def __enter__(self):
        print("Entering context")
        # return value is bound to 'as' variable
        return self.val
        
    def __exit__(self, exctype, value, tb):
        # Potentially handle an exception in the body
        if exctype is not None:
            print(f"CM body raised {exctype.__name__}({value})")
            print(f"Inspect {tb}")
        # Suppress propagation of exception by returning True
        print("Exiting context")
        return True

In [8]:
with MyContext() as t:
    print("The answer is", t)

Initializing context
Entering context
The answer is 42
Exiting context


In [9]:
with MyContext(33) as t:
    x = t/0
    print(x)

Initializing context
Entering context
CM body raised ZeroDivisionError(division by zero)
Inspect <traceback object at 0x0000014317AC2700>
Exiting context


Although `MyContext` simply returns a value as the context object (in the example, an integer), a very common pattern is to `return self` and provide other APIs of the class instance as methods of the context object.  For example, `open()` file handles do this, and thereby also implement other protocols, such as the Iterator Protocol.

# Exercises

## Description

The Python `timeit` module provides very good quality timing of operations, with statistics, disabling of garbage collection, and options to run separate setup versus timing code.  Within Jupyter or IPython, the `%time` and `%timeit` magics offer similar bundled capability.

In this exercise, you will create something less sophisticated than that, but still useful.  You should develop a custom context manager called `Timer` that will run code multiple times, and save both the result from each run and the time it took.  For example, using the `slow_random_normal()` function provided in the Setup:

```python
>>> with Timer(slow_random_normal, loops=5) as fn:
...     fn.run(-3, stdev=5)

>>> print("Timers:", fn.timers)
Timers: [0.0501605, 0.0502064, 0.0501945, 0.0502957, 0.0501452]
>>> print("Results:", fn.results)
Results: [-0.2525025, -1.4776930, -2.4067729, -1.3618136, -0.6298354]
```

Note that the test cases assume `slow_random_normal()` retains the same implementation provided.

## Setup

In [10]:
from time import sleep
from random import random

def slow_random_normal(mean=0, stdev=1):
    sleep(0.05)
    r = mean + stdev*random()
    return r

class Timer:
    "Implementation here"

## Solution

In [11]:
from time import perf_counter

class Timer:
    def __init__(self, fn=lambda *args, **kws: None, loops=1):
        self.loops = loops
        self.fn = fn
        self.timers = list()
        self.results = list()
        
    def __enter__(self):
        return self
    
    def __exit__(self, type_, value, tb):
        pass
    
    def run(self, *args, **kws):
        for _ in range(self.loops):
            start = perf_counter()
            result = self.fn(*args, **kws)
            self.timers.append(perf_counter() - start)
            self.results.append(result)       

## Test Cases

In [3]:
def test_is_cm():
    assert hasattr(Timer, '__enter__')
    assert hasattr(Timer, '__exit__')
    
test_is_cm()

In [5]:
def test_is_timing():
    from math import isclose
    from statistics import mean
    with Timer(slow_random_normal, loops=100) as fn:
        fn.run(-5, stdev=0.1)
    assert len(fn.timers) == len(fn.results) == 100
    assert isclose(mean(fn.timers), 0.05, abs_tol=0.01)
    assert isclose(mean(fn.results), -5, abs_tol=0.1)
    
test_is_timing()

-------------
Materials licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by the authors